In [89]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

offers_recieved = pd.read_csv('for_model.csv')
#offers_recieved = offers_recieved.drop(['id_x', 'id_y','value','person','offer_id','channels','influence_start',
#                                       'influence_end','Unnamed: 0','event','offer_type','index'], axis=1)
offers_recieved.info()

transcript = pd.read_json('data/transcript.json', orient='records', lines=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17189 entries, 0 to 17188
Data columns (total 19 columns):
Unnamed: 0         17189 non-null int64
offer_type         17189 non-null object
time               17189 non-null int64
difficulty         17189 non-null int64
duration           17189 non-null int64
reward             17189 non-null int64
web                17189 non-null int64
email              17189 non-null int64
mobile             17189 non-null int64
social             17189 non-null int64
bogo               17189 non-null int64
discount           17189 non-null int64
informational      17189 non-null int64
age                17189 non-null int64
income             17189 non-null float64
member_date_int    17189 non-null int64
F                  17189 non-null int64
M                  17189 non-null int64
offer_impact       17189 non-null float64
dtypes: float64(2), int64(16), object(1)
memory usage: 2.5+ MB


In [90]:
print(offers_recieved.describe())
print(offers_recieved.head())

         Unnamed: 0          time    difficulty      duration        reward  \
count  17189.000000  17189.000000  17189.000000  17189.000000  17189.000000   
mean   38318.052243    334.436209      7.708651      6.778114      4.523474   
std    21914.941331    195.303471      3.871713      2.060603      3.412746   
min        0.000000      0.000000      0.000000      3.000000      0.000000   
25%    19311.000000    168.000000      5.000000      5.000000      2.000000   
50%    38472.000000    408.000000     10.000000      7.000000      3.000000   
75%    57272.000000    504.000000     10.000000      7.000000      5.000000   
max    76271.000000    576.000000     20.000000     10.000000     10.000000   

                web    email        mobile        social          bogo  \
count  17189.000000  17189.0  17189.000000  17189.000000  17189.000000   
mean       0.794462      1.0      0.971493      0.792018      0.422247   
std        0.404106      0.0      0.166420      0.405876      0.49

In [91]:
offers_recieved = offers_recieved.drop(['Unnamed: 0','offer_type'],axis=1)

In [99]:
X, y = offers_recieved.iloc[:,:-1],offers_recieved.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)



rmse = []
for i in range(1,10):
    for j in range(100,400,50):
        xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                    max_depth = i, alpha = 7, n_estimators = j)
        xg_reg.fit(X_train,y_train)
        test_preds = xg_reg.predict(X_test)
        train_preds = xg_reg.predict(X_train)
        test_rmse = np.sqrt(mean_squared_error(y_test, test_preds))
        train_rmse = np.sqrt(mean_squared_error(y_train, train_preds))
        rmse.append([i,j,train_rmse,test_rmse])
    
rmse = pd.DataFrame(rmse)
rmse.to_csv('rmse.csv')
    

    

In [100]:
#proof of concept
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 3, alpha = 6, n_estimators = 350)

xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

preds = xg_reg.predict(X)

visual = offers_recieved
visual['preds'] = preds
visual.to_csv('visual.csv')

RMSE: 0.014040
